In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
%matplotlib inline
warnings.filterwarnings(action='ignore')

In [ ]:
## to view all the columns in the 
pd.set_option("display.max_columns", None)

In [ ]:
df = pd.read_csv('fifa.csv')
df.head()

In [ ]:
## loading the variable information csv file.
data_info = pd.read_csv('fifa_ variable_information.csv')
data_info.head()

In [ ]:
## QUERY 2
df[['Photo','Flag','Club Logo','Body Type','Jersey Number']].head(2)

In [ ]:
## dropping the above five categorical columns, which are redundant for our analysis.
df.drop(['Photo','Flag','Club Logo','Body Type','Jersey Number'],axis=1,inplace=True)

In [ ]:
df.shape

In [ ]:
## QUERY 3
## Just checking some samples.
df[['Value','Wage','Release Clause']].sample(4)

In [ ]:
## Writing a function to clean the data of columns "Value" and "Wage"
def convert(entry):
    
    if entry==np.nan:
        new_entry=float(0)
    else:
        entry=entry.replace('€','')
        if entry[-1]=='K':
            new_entry=float(entry.replace('K',''))*1000
        else:
            new_entry=float(entry.replace('M',''))*1000000
    return new_entry

In [ ]:
## applying the above function to column 'Value'
df['Value']= df['Value'].apply(lambda x : convert(x))

## applying the above function to column 'Wage'
df['Wage'] = df['Wage'].apply(lambda x: convert(x))

## applying the above function to column 'Release Clause' 
df['Release Clause'] = df['Release Clause'].fillna('0').apply(lambda x: convert(x))

In [ ]:
## Just checking some samples after cleaning the data.
df[['Value','Wage','Release Clause']].sample(4)

In [ ]:
## QUERY 4
## just checking some samples
df['Joined'].head()

In [ ]:
## Keeping only year from the column 'Joined'
## Filling null by 0 and converting it to integer
df['Joined'] = df['Joined'].str.split().str.get(2).fillna(0).astype(int)

In [ ]:
## just check some samples after cleaning.
df['Joined'].head()

In [ ]:
## QUERY 5
## checking the unique entries
df['Contract Valid Until'].unique()

In [ ]:
## converting into pandas datetime format.
df['Contract Valid Until'] = pd.to_datetime(df['Contract Valid Until'])

In [ ]:
## checking the data after conversion.
df['Contract Valid Until'].sample(5)

In [ ]:
## QUERY 6
df['Height'].head()

In [ ]:
## writing a function
def height_convert(x):
    if x is not None:
        return float(str(x).replace("'",'.'))

In [ ]:
## applying a function
df['Height'] = df['Height'].apply(height_convert)

In [ ]:
df['Height'].head()

In [ ]:
## QUERY 7
df['Weight'].head()

In [ ]:
df['Weight'] = df['Weight'].fillna('0').apply(lambda x: x.replace('lbs','')).astype(float)

In [ ]:
df['Weight'].head()

In [ ]:
## QUIERY 8
df.isnull().sum()/len(df)*100

In [ ]:
## The column 'Loaned_from' has 93% of missing values, we will drop that column
df.drop('Loaned From',axis=1,inplace=True)

In [ ]:
missing_values = df.isnull().sum()/len(df)

## These are the index of both categorical columns as well as numerical columns having null values.
index = [i  for i in missing_values[missing_values>0].index]

## separating numerical columns and categorical columns
numerical_columns = [i     for i in index      if i in df.columns and  df[i].dtype!=object]
categorical_columns = [i     for i in index      if i in df.columns and  df[i].dtype==object]

## Filling the null values of the numerical columns with median computation.
for columns in numerical_columns:
    df[columns]=df[columns].fillna(df[columns].median())
    
## Filling the null values of the categorical columns with mode computation.
for columns in categorical_columns:
    df[columns]=df[columns].fillna(df[columns].mode()[0])

In [ ]:
df.isnull().sum()

In [ ]:
## QUERY 9
sns.distplot(df['Overall'])
plt.show()

In [ ]:
## QUERY 10
top20_players = df.groupby('Name')['Overall'].max().sort_values(ascending=False).head(20)
top20_players

In [ ]:
## QUERY 11
Top20_df = df[ (df['Name'].isin(top20_players.index)) & (df['Overall'].isin(top20_players.values)) ]
Top20_df.head()

In [ ]:
## QUERY 12
Top20_df[['Age','Wage']].mean()

In [ ]:
## QUERY 13
Top20_df[Top20_df['Wage']==Top20_df['Wage'].max()][['Name','Wage']]

In [ ]:
## QUERY 14
## creating a dataframe
df_14 = df[['Name','Club','Wage','Overall']]
df_14.head(2)

In [ ]:
## Finding the average overall rating for each club
df_14.groupby('Club')['Overall'].mean()

In [ ]:
## Displaying the average overall rating of Top10 Clubs using a bar plot
df_14.groupby('Club')['Overall'].mean().sort_values(ascending=False).head(10).plot(kind='bar')
plt.show()

In [ ]:
## QUERY 15
plt.figure(figsize=(18,5))
sns.boxplot(df['Age'],df['Potential'])
plt.show()

In [ ]:
## QUERY 16
imp_features = df[['Potential','Overall','Value','International Reputation','Release Clause','Wage', ]]

sns.heatmap(imp_features.corr(),annot=True,cmap='viridis')
plt.show()

In [ ]:
sns.pairplot(imp_features)
plt.show()

In [ ]:
## QUERY 17
df['Position'].value_counts().sort_values(ascending=False).head(1)

In [ ]:
df['Position'].value_counts().sort_values(ascending=True).head(1)

In [ ]:
plt.figure(figsize=(18,5))
sns.histplot(df['Position'])
plt.show()

In [ ]:
## QUERY 18
df[(df['Club']=='Juventus') & (df['Wage']>200000)]

In [ ]:
## QUERY 19
top5 = df.groupby(['Position']).apply(lambda x : x.sort_values('Overall',ascending=False).head(5))
top5.head(10)

In [ ]:
## QUERY 20
top5['Wage'].groupby('Position').mean()